In [1]:
'''
Marginaising out discrete parameters

Stan doesn't diretly allow discrete parameters but they can be maginalised
=> Sum the joint density over all possible values (on a log scale with log_sum_exp)
'''

import pystan as stan
import numpy as np

nStudy=20
N=10
Z=np.ones((N,nStudy))
Z=Z.astype(int)
theta=[0.1,0.9]
state=np.zeros((nStudy))

for i in range(nStudy):
    if np.random.uniform()>0.5:
        Z[:,i]=np.random.binomial(1,theta[0],N)
        state[i]=1
    else:
        Z[:,i]=np.random.binomial(1,theta[1],N)
        state[i]=0

X=sum(Z)

#Stan model
model = stan.StanModel(file='marginalise_discrete_vars.stan')
fit = model.sampling(data={'N':N,'X':X,'nStudy':nStudy},iter=200,chains=4)
print(fit)


#Check if the model correctly predicted state 1 if prob>0.5
extracted = fit.extract()
pstate=extracted['pstate']
state_est=np.where(np.mean(pstate,0)>0.5,1,0)
print(state_est)
#state=state+1
state=state.astype(int)
print(state)
print(np.sum(state_est-state))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8374a6545d3eb70a7daf8215e71d4a42 NOW.


Inference for Stan model: anon_model_8374a6545d3eb70a7daf8215e71d4a42.
4 chains, each with iter=200; warmup=100; thin=1; 
post-warmup draws per chain=100, total post-warmup draws=400.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[1]    -2.34    0.03    0.3  -3.02  -2.52  -2.35  -2.14  -1.78    123   1.03
alpha[2]     2.24    0.02   0.43   1.47   1.94    2.2   2.53   3.11    428   0.99
theta[1]     0.09  2.4e-3   0.02   0.05   0.07   0.09   0.11   0.14    108   1.03
theta[2]      0.9  1.8e-3   0.04   0.81   0.87    0.9   0.93   0.96    446   0.99
lp[1,1]     -1.65    0.03   0.27  -2.25  -1.81   -1.6  -1.46  -1.17    106   1.03
lp[2,1]    -16.54    0.19   2.14 -21.52 -17.85  -16.6  -15.1 -12.68    123   1.03
lp[3,1]     -1.65    0.03   0.27  -2.25  -1.81   -1.6  -1.46  -1.17    106   1.03
lp[4,1]     -1.69  6.3e-3   0.07   -1.9   -1.7  -1.66  -1.65  -1.64    108   1.01
lp[5,1]     -3.88    0.06   0.64  -5.44  -4.25  -3.87  -3.43   -2.8    124   